In [1]:

from scipy.stats import norm

from tensorflow.keras.datasets import mnist

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from tensorflow.keras import Model, Sequential
from tensorflow.keras.losses import binary_crossentropy
import matplotlib.pyplot as plt



original_dim = 27
intermediate_dim = 12
latent_dim = 2
batch_size = 10
epochs = 5
epsilon_std = 1.0

In [2]:
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """
    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

class KLDivergenceLayer(Layer):
    """ Identity transform layer that adds KL divergence to the final model loss."""
    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)
    def call(self, inputs):
        mu, log_var = inputs
        kl_batch = - .5 * K.sum(1+log_var-K.square(mu)-K.exp(log_var), axis=-1)
        self.add_loss(K.mean(kl_batch), inputs=inputs)
        return inputs
    
decoder = Sequential([Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),Dense(original_dim, activation='sigmoid')])

In [3]:
x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)
z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)
z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(stddev=epsilon_std,shape=(K.shape(x)[0], latent_dim)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])
x_pred = decoder(z)
vae = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='rmsprop', loss=nll, metrics=['accuracy'])

In [4]:
import pickle
with open('ae_drop.txt','rb') as f:
    data=pickle.load(f)

In [5]:
# train the VAE on MNIST digits

vae.fit(data,data,shuffle=True,epochs=epochs,batch_size=batch_size,validation_data=(data, data))
encoder = Model(x, z_mu)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 21526 samples, validate on 21526 samples
Epoch 1/5
21526/21526 [==============================] - 8s 359us/sample - loss: 12.3664 - accuracy: 0.3938 - val_loss: 11.8520 - val_accuracy: 0.4034
Epoch 2/5
21526/21526 [==============================] - 7s 337us/sample - loss: 11.8462 - accuracy: 0.4034 - val_loss: 11.8395 - val_accuracy: 0.4034
Epoch 3/5
21526/21526 [==============================] - 6s 301us/sample - loss: 11.8402 - accuracy: 0.4034 - val_loss: 11.8360 - val_accuracy: 0.4034
Epoch 4/5
21526/21526 [==============================] - 7s 310us/sample - loss: 11.8348 - accuracy: 0.4035 - val_loss: 11.8347 - val_accuracy: 0.4035
Epoch 5/5
21526/21526 [==============================] - 6s 289us/sample - loss: 11.8321 - accuracy: 0.4038 - val_loss: 11.8336 - val_accuracy: 0.4044


In [6]:
#         display a 2D plot of the digit classes in the latent space
z_test = encoder.predict(data, batch_size=batch_size)
#plt.figure(figsize=(6, 6))
#plt.scatter(z_test[:, 0], z_test[:, 1],alpha=.4, s=3**2, cmap='viridis')
#plt.colorbar()
#plt.show()
z_test[:,0]

array([-0.03009982, -0.02983421, -0.03090478, ..., -0.00920495,
        0.01520837, -0.00609072], dtype=float32)

In [7]:
z_test[:,1]

array([0.24911702, 0.24339417, 0.2558358 , ..., 0.1519604 , 0.16295813,
       0.27407905], dtype=float32)

In [8]:
with open('z_test_ae_drop.txt','wb') as f:
    pickle.dump(z_test,f)


In [ ]:
plt.scatter(z_test[:,0],z_test[:,1])
plt.colorbar()
plt.show()